In [2]:
import os
from queue import PriorityQueue
from time import time
import random
import string

In [3]:
def count_time(func, *args):
    start_time = time()
    func(*args)
    return time() - start_time

def count_letters_in_file(filename):
    letters = dict()
    with open(filename, "r", encoding="UTF-8") as file:
        text = file.read()
        for letter in text:
            if letter in letters:
                letters[letter] += 1
            else:
                letters[letter] = 1
    return letters

def build_hoffman_tree(letters):

    class Node:
        def __init__(self):
            self.value = None
            self.left = None
            self.right = None

        def __lt__(self, other):
            return 0

        def __len__(self):
            return 0

        def __gt__(self, other):
            return 0

        def __ge__(self, other):
            return 0

        def __cmp__(self, other):
            return 0

        def __eq__(self, other):
            return 0

        def __ne__(self, other):
            return 0

    def build_coder_dict(node: Node, string = ""):
        if type(node.left) == Node:
            build_coder_dict(node.left, string + "0")
        else:
            decoder[node.left] = string + "0"
        if type(node.right) == Node:
            build_coder_dict(node.right, string + "1")
        else:
            decoder[node.right] = string + "1"


    sorted_letters = PriorityQueue(len(letters))
    for letter in letters:
        sorted_letters.put((letters[letter], ord(letter), letter))


    while sorted_letters.qsize() != 1:
        letter1 = sorted_letters.get()
        letter2 = sorted_letters.get()
        new_node = Node()
        new_node.value = letter1[0] + letter2[0]
        new_node.right = letter1[2]
        new_node.left = letter2[2]
        sorted_letters.put((new_node.value, float("inf"), new_node))

    tree = sorted_letters.get()[2]
    decoder = dict()
    build_coder_dict(tree)
    # print(decoder)
    return decoder

def compress_file_using_decoder(filename, decoder):
    with open(filename, "r", encoding="UTF-8") as file_to_decode:
        text = file_to_decode.read()
        s = []
        for letter in text:
            s.append(decoder[letter])
        s = "".join(s)

        # make s divided by 8
        fill = 8 - len(s) % 8
        temp = "0" * fill
        temp += "0" * (8 - fill.bit_length()) + bin(fill).replace("0b", "")
        s += temp

        # print(s)
        # print(len(s))
        with open(f"coded_{filename[:-4]}.bin", "wb") as coded_file:
            b = bytearray()
            for x in range(0, len(s), 8):
                # b.append(to_bin(s[x:x+8]))
                a = int(s[x:x+8], 2)
                # print(a)
                b.append(a)  # bytearray appenduje liczbe w systemie 10, a jak to robi, to juz nie moj problem
            coded_file.write(b)


def reverse_decoder(decoder):
    new_decoder = dict()
    for key in decoder.keys():
        new_decoder[decoder[key]] = key
    return new_decoder



In [4]:
def decode_using_decored(filename, decoder):

    # print(decoder)
    with open(filename, "rb") as file_to_decode:
        text = file_to_decode.read()
        decoded_text = []
        step = ""
        for i in range(len(text) - 2):
            bajt = text[i]
            x = bin(bajt).replace("0b", "")
            x = ("0" * (8 - len(str(x))) + str(x))
            # print(x)
            for y in x:
                step += y
                # print(step)
                if sign := decoder.get(step):
                    decoded_text.append(sign)
                    step = ""
        last_bajt_to_check = bin(text[-2]).replace("0b", "")
        for i in range(8 - i):
            step += last_bajt_to_check[i]
            if sign := decoder.get(step):
                decoded_text.append(sign)
                step = ""
        # print("XD", additional_bits)
        # print(decoded_text)

        with open(f"decoded_{filename[:-4]}.txt", "w", encoding="UTF=8") as decoded_file:
            decoded_file.write("".join(decoded_text))





In [5]:
def code_and_decode_file(filename):

    letters = count_letters_in_file(filename)
    decoder = build_hoffman_tree(letters)
    compress_file_using_decoder(filename, decoder)
    decode_using_decored(f"coded_{filename[:-4]}.bin", reverse_decoder(decoder))

def compare_file(filename):
    before = os.stat(filename).st_size
    after = os.stat(f"coded_{filename[:-4]}.bin").st_size
    return after / before

def generate_random_asci_file(filename, n):

    random_string = ''.join(random.choices(string.printable, k=n))

    with open(filename, "w", encoding="UTF-8") as file:
        file.write(random_string)

In [62]:
filename = "gutenberg1kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 55.935%. It took 0.01 seconds


In [63]:
filename = "gutenberg10kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 55.648%. It took 0.02 seconds


In [64]:
filename = "gutenberg100kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 55.54%. It took 0.14 seconds


In [65]:
filename = "gutenberg1MB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 55.373%. It took 1.45 seconds


In [66]:
filename = "linux1kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 57.798%. It took 0.0 seconds


In [67]:
filename = "linux10kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 64.531%. It took 0.02 seconds


In [68]:
filename = "linux100kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 63.548%. It took 0.15 seconds


In [69]:
filename = "linux1MB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 62.083%. It took 2.03 seconds


# Generate random signs in files

In [79]:
filename = "randomsigns1kB.txt"
generate_random_asci_file(filename, 1000)

In [80]:
filename = "randomsigns10kB.txt"
generate_random_asci_file(filename, 10000)

In [84]:
filename = "randomsigns100kB.txt"
generate_random_asci_file(filename, 101000)

In [90]:
filename = "randomsigns1MB.txt"
generate_random_asci_file(filename, 1038000)

In [91]:
filename = "randomsigns1kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 82.008%. It took 0.01 seconds


In [92]:
filename = "randomsigns10kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 82.456%. It took 0.03 seconds


In [93]:
filename = "randomsigns100kB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 82.823%. It took 0.18 seconds


In [94]:
filename = "randomsigns1MB.txt"
t = count_time(code_and_decode_file, filename)
print(f"File was compressed by {round(compare_file(filename) * 100, 3)}%. It took {round(t, 2)} seconds")

File was compressed by 82.875%. It took 2.46 seconds
